<a href="https://colab.research.google.com/github/Zahab163/ML_notes/blob/main/Titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import learning_curve, validation_curve
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")

run without csv file , so it occur error

In [2]:
# Load the data
train_df = pd.read_csv('/content/train.csv')

print("Dataset Shape:", train_df.shape)
print("\nFirst 5 rows:")
train_df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
print("\nDataset Info:")
print(train_df.info())
print("\nMissing Values:")
print(train_df.isnull().sum())

In [ ]:
# Exploratory Data Analysis
plt.figure(figsize=(20, 15))



In [ ]:

# 1. Survival distribution
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 1)
sns.countplot(x='Survived', data=train_df)
plt.title('Survival Distribution')
plt.show()

In [ ]:
# 2. Survival by gender
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 2)
sns.countplot(x='Survived', hue='Sex', data=train_df)
plt.title('Survival by Gender')
plt.show()


In [ ]:
# 3. Survival by class
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 3)
sns.countplot(x='Survived', hue='Pclass', data=train_df)
plt.title('Survival by Class')
plt.show()


In [ ]:

# 4. Age distribution by survival
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 4)
sns.histplot(data=train_df, x='Age', hue='Survived', element='step', stat='density', common_norm=False)
plt.title('Age Distribution by Survival')
plt.show()


In [ ]:

# 5. Fare distribution by survival
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 5)
sns.histplot(data=train_df, x='Fare', hue='Survived', element='step', stat='density', common_norm=False)
plt.title('Fare Distribution by Survival')
plt.show()

In [ ]:

# 6. Embarkment port vs survival
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 6)
sns.countplot(x='Embarked', hue='Survived', data=train_df)
plt.title('Survival by Embarkment Port')
plt.show()

In [ ]:

# 7. SibSp vs survival
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 7)
sns.countplot(x='SibSp', hue='Survived', data=train_df)

plt.title('Survival by SibSp')
plt.show()

In [ ]:
# 8. Parch vs survival
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 8)
sns.countplot(x='Parch', hue='Survived', data=train_df)
plt.title('Survival by Parch')
plt.show()


In [ ]:

# 9. Correlation heatmap
plt.figure(figsize=(20, 15))
plt.subplot(3, 4, 9)
numeric_df = train_df.select_dtypes(include=[np.number])
numeric_df = numeric_df.dropna()
corr = numeric_df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')

plt.tight_layout()
plt.show()

In [ ]:
# Feature Engineering
def engineer_features(df):
    df_engineered = df.copy()

    # Extract title from name
    df_engineered['Title'] = df_engineered['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df_engineered['Title'] = df_engineered['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don',
                                                            'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df_engineered['Title'] = df_engineered['Title'].replace('Mlle', 'Miss')
    df_engineered['Title'] = df_engineered['Title'].replace('Ms', 'Miss')
    df_engineered['Title'] = df_engineered['Title'].replace('Mme', 'Mrs')

    # Fill missing ages with median based on title and class
    df_engineered['Age'] = df_engineered.groupby(['Title', 'Pclass'])['Age'].transform(
        lambda x: x.fillna(x.median()))

    # Fill missing embarked with mode
    df_engineered['Embarked'].fillna(df_engineered['Embarked'].mode()[0], inplace=True)

    # Drop unnecessary columns
    df_engineered.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

    # Encode categorical variables using one-hot encoding
    df_engineered_encoded = pd.get_dummies(df_engineered, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


    return df_engineered_encoded

In [ ]:
train_df_engineered = engineer_features(train_df)
print("\nMissing values after engineering:")
print(train_df_engineered.isnull().sum())

In [ ]:

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the models
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, "predict_proba") else None

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Perform cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')

    # Store results
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

    # Print results
    print(f"{name}:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  CV Mean Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print()

In [ ]:
# Compare model performance
model_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in model_names]
cv_means = [results[name]['cv_mean'] for name in model_names]
cv_stds = [results[name]['cv_std'] for name in model_names]

x_pos = np.arange(len(model_names))

plt.figure(figsize=(12, 6))
plt.bar(x_pos, accuracies, yerr=cv_stds, align='center', alpha=0.7, ecolor='black', capsize=10)
plt.ylabel('Accuracy')
plt.title('Model Comparison')
plt.xticks(x_pos, model_names, rotation=45)
plt.ylim(0.7, 0.9)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:

# Detailed evaluation for each model
for name in results:
    print(f"\n{name} - Classification Report:")
    print(classification_report(y_test, results[name]['y_pred']))

    # Confusion matrix
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(y_test, results[name]['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
# ROC Curves for models that support probability predictions
plt.figure(figsize=(10, 8))
for name in results:
    if results[name]['y_pred_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_test, results[name]['y_pred_proba'])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# Precision-Recall Curves
plt.figure(figsize=(10, 8))
for name in results:
    if results[name]['y_pred_proba'] is not None:
        precision, recall, _ = precision_recall_curve(y_test, results[name]['y_pred_proba'])
        plt.plot(recall, precision, lw=2, label=f'{name}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc="lower left")
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Learning curves for the best model
best_model_name = max(results, key=lambda x: results[x]['cv_mean'])
best_model = results[best_model_name]['model']

plt.figure(figsize=(10, 6))
train_sizes, train_scores, test_scores = learning_curve(
    best_model, X_train_scaled, y_train, cv=5,
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, 'o-', color='r', label='Training score')
plt.plot(train_sizes, test_mean, 'o-', color='g', label='Cross-validation score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='r')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='g')


plt.xlabel('Training examples')
plt.ylabel('Accuracy')
plt.title(f'Learning Curve for {best_model_name}')
plt.legend(loc='best')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Feature importance for tree-based models
plt.figure(figsize=(15, 10))
for i, name in enumerate(['Random Forest', 'Gradient Boosting']):
    if name in results:
        model = results[name]['model']
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            indices = np.argsort(importance)[::-1]

            plt.subplot(2, 2, i+1)
            plt.bar(range(X.shape[1]), importance[indices])
            plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
            plt.title(f'Feature Importance - {name}')
            plt.tight_layout()

plt.show()


In [ ]:
# Validation curve for Random Forest (n_estimators parameter)
if 'Random Forest' in results:
    param_range = np.arange(10, 201, 20)
    train_scores, test_scores = validation_curve(
        RandomForestClassifier(random_state=42), X_train_scaled, y_train,
        param_name="n_estimators", param_range=param_range,
        cv=5, scoring="accuracy"
    )

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(param_range, train_mean, label="Training score", color="r")
    plt.plot(param_range, test_mean, label="Cross-validation score", color="g")
    plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color="r")
    plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, alpha=0.1, color="g")
    plt.legend(loc="best")
    plt.xlabel("Number of Estimators")
    plt.ylabel("Accuracy")
    plt.title("Validation Curve for Random Forest")
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
# Print final comparison table
comparison_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': accuracies,
    'CV Mean Accuracy': cv_means,
    'CV Std': cv_stds
}).sort_values('CV Mean Accuracy', ascending=False)

print("\nFinal Model Comparison:")
print(comparison_df.to_string(index=False))